# **Extract SALT2-simulated SNe Ia samples**

SN simulations created with SNANA.

The ZP are 27.5 by default (do not pay attention to the ZEROPT column).

PHOTFLAG_DETECT: 4096 # set this bit for each detection

# **=======================================================================**
# **My Simulations**
___

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
import random
import os

In [2]:
def compare_salt2(hdul_head_file, fits_file, sn_name):
    hdul_head = fits.open(hdul_head_file)
    fit_params_df = pd.read_csv(fits_file, delim_whitespace=True, comment='#')
    column_names = hdul_head[1].data.dtype.names
    
    index_dict = {}
    for i, col_name in enumerate(column_names):
        index_dict[col_name] = i

    sn_dict = {}
    for sn_info in hdul_head[1].data:
        if sn_name == sn_info[index_dict['SNID']]:
            sn_dict[sn_name] = {col_name:sn_info[index_dict[col_name]] for col_name in column_names}
            
    sim_params_df = pd.DataFrame.from_dict(sn_dict, 'index').sort_values('SNID')
    print(f'SN: {sn_name}\n')
    ######################################################################
    sim_mb = sim_params_df[sim_params_df.SNID==sn_name]['SIM_SALT2mB'].values[0]
    sim_x1 = sim_params_df[sim_params_df.SNID==sn_name]['SIM_SALT2x1'].values[0]
    sim_c = sim_params_df[sim_params_df.SNID==sn_name]['SIM_SALT2c'].values[0]
    sim_tmax = sim_params_df[sim_params_df.SNID==sn_name]['SIM_PEAKMJD'].values[0]
    print(f'Simulated SALT2 params --- mb: {sim_mb:.3f}, x1: {sim_x1:.3f},',
          f'dm15: {1.09-0.161*sim_x1+0.13*sim_x1**2-0.0013*sim_x1**3:.3f}, c: {sim_c:.3f}, tmax: {sim_tmax:.3f}')

    mb = fit_params_df[fit_params_df.CID.astype(str)==sn_name]['mB'].values[0]
    x1 = fit_params_df[fit_params_df.CID.astype(str)==sn_name]['x1'].values[0]
    c = fit_params_df[fit_params_df.CID.astype(str)==sn_name]['c'].values[0]
    tmax = fit_params_df[fit_params_df.CID.astype(str)==sn_name]['PKMJD'].values[0]
    print(f'Fitted SALT2 params    --- mb: {mb:.3f}, x1: {x1:.3f},',
          f'dm15: {1.09-0.161*x1+0.13*x1**2-0.0013*x1**3:.3f}, c: {c:.3f}, tmax: {tmax:.3f}')

In [15]:
def extract_sample(hdul_head_file, hdul_phot_file, mag_sys, sn_name_prefix=''):
    
    lowz_filters = {'a':'4shooter2_U', 'b':'4shooter2_B', 'c':'4shooter2_V', 'd':'4shooter2_R', 'e':'4shooter2_I',
                'f':'p1_U', 'g':'p1_B', 'h':'p1_V', 'i':'p1_r', 'j':'p1_i',
                'k':'p1_U', 'l':'p1_B', 'm':'p1_V', 'n':'p1_r', 'o':'p1_i',
                'p':'p2_B', 'q':'p2_V', 'r':'p2_r', 's':'p2_i',
                't':'csp_u', 'u':'csp_B', 'v':'csp_o', 'w':'csp_m', 
                'x':'csp_n', 'y':'csp_g', 'z':'csp_r', 'A':'csp_i'}
    
    bands_dict = {'CFA3S':['a', 'b', 'c', 'd', 'e'],
              'CFA3K':['f', 'g', 'h', 'i', 'j'],
              'CFA4p1':['k', 'l', 'm', 'n', 'o'],
              'CFA4p2':['p', 'q', 'r', 's'],
              'CSP':['t', 'u', 'v', 'w', 'x', 'y', 'z', 'A']
             }
    
    #######################################
 
    ####### HEAD ######
    hdul_head = fits.open(hdul_head_file)
    column_names = hdul_head[1].data.dtype.names
    
    index_dict = {}
    for i, col_name in enumerate(column_names):
        index_dict[col_name] = i

    sn_dict = {}
    for sn_info in hdul_head[1].data:
        sn_name = sn_info[index_dict['SNID']]
        sn_dict[sn_name] = {col_name:sn_info[index_dict[col_name]] for col_name in column_names}

    ####### PHOT ######
    hdul_lc = fits.open(hdul_phot_file)
    column_names = hdul_lc[1].data.dtype.names
    
    index_dict = {}
    for i, col_name in enumerate(column_names):
        index_dict[col_name] = i
    
    sn_names = [sn for sn in sn_dict.keys()]
    for sn_name in sn_names:

        # sn info
        imin = sn_dict[sn_name]['PTROBS_MIN'] - 1
        imax = sn_dict[sn_name]['PTROBS_MAX']
        ra = sn_dict[sn_name]['RA']
        dec = sn_dict[sn_name]['DEC']
        z = sn_dict[sn_name]['REDSHIFT_FINAL']

        sn_lc = hdul_lc[1].data[imin:imax]
        sn_lc = np.array(list(set(sn_lc))).T  # turn tuplets into array

        # turn the SN info into a pandas dataframe
        lc_indexes = [i for i in index_dict.values()]
        column_names = [name for name in index_dict.keys()]
        lc_df = pd.DataFrame(data=sn_lc[lc_indexes].T, columns=column_names)
        lc_df = lc_df.rename(columns={'MJD':'mjd', 'FLT':'band', 'FLUXCAL':'flux', 'FLUXCALERR':'flux_err'})

        lc_df['zp'] = 27.5  # from SNANA manual / e.g., page 74
        lc_df['mag_sys'] = mag_sys
        
        #############################
        if 'lowz' in hdul_head_file.lower():
            # use only the filters for the survey specified in the SUBSURVEY key
            subsurvey = sn_dict[sn_name]['SUBSURVEY']
            lc_df = lc_df[lc_df.band.isin(bands_dict[subsurvey])]  
            for band in lc_df.band.unique():
                lc_df.loc[lc_df.band==band, 'band'] = lowz_filters[band]
            
        elif 'sdss' in hdul_head_file.lower():
            lc_df['band'] = 'sdss_' + lc_df['band'].astype(str)
        elif 'snls' in hdul_head_file.lower():
            lc_df['band'] = 'Megacam_' + lc_df['band'].astype(str)
        elif 'ps1' in hdul_head_file.lower():
            lc_df['band'] = 'ps1_' + lc_df['band'].astype(str)    
        elif 'des' in hdul_head_file.lower():
            lc_df['band'] = 'des_' + lc_df['band'].astype(str)
        ##############################
        lc_df[['flux', 'flux_err']] = lc_df[['flux', 'flux_err']].round(3)
        
        sn_file_name = f'data_simulations/{sn_name_prefix}{sn_name}.dat'
        with open(sn_file_name, 'w') as file:
            file.write('name z ra dec\n')
            file.write(f'{sn_name_prefix}{sn_name} {z:.5f} {ra:.3f} {dec:.3f} \n')
        lc_df[['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']].to_csv(sn_file_name, index=False, sep=' ', mode='a')

    print(f'{len(sn_names)} SNe extracted!')

In [4]:
def print_sim_mags(hdul_head_file, sn_name, bands_list):
    
    lowz_filters = {'a':'4shooter2_U', 'b':'4shooter2_B', 'c':'4shooter2_V', 'd':'4shooter2_R', 'e':'4shooter2_I',
                'f':'p1_U', 'g':'p1_B', 'h':'p1_V', 'i':'p1_r', 'j':'p1_i',
                'k':'p1_U', 'l':'p1_B', 'm':'p1_V', 'n':'p1_r', 'o':'p1_i',
                'p':'p2_B', 'q':'p2_V', 'r':'p2_r', 's':'p2_i',
                't':'csp_u', 'u':'csp_B', 'v':'csp_o', 'w':'csp_m', 
                'x':'csp_n', 'y':'csp_g', 'z':'csp_r', 'A':'csp_i'}
    
    #if 'lowz' in hdul_head_file.lower():
    #    bands_list = [lowz_filters[letter] for letter in bands_list]
    
    hdul_head = fits.open(hdul_head_file)
    column_names = hdul_head[1].data.dtype.names
    
    index_dict = {}
    for i, col_name in enumerate(column_names):
        index_dict[col_name] = i

    sn_dict = {}
    for sn_info in hdul_head[1].data:
        if sn_name == sn_info[index_dict['SNID']]:
            sn_dict[sn_name] = {col_name:sn_info[index_dict[col_name]] for col_name in column_names}
            
    sim_params_df = pd.DataFrame.from_dict(sn_dict, 'index')#.sort_values('SNID')

    print(f'SN: {sn_name}\n')
    for band in bands_list:
        print(band, '=',sim_params_df[sim_params_df.SNID.astype(str)==sn_name][f'SIM_PEAKMAG_{band}'].values[0])

In [22]:
def delete_files(prefix, cad):
    
    flist = [file for file in glob.glob(f'data_simulations/{prefix}{cad}dcad*')]
    for f in flist:
        os.remove(f)

___
# **Low-z**

In [36]:
#cadences = [1, 3, 5, 7]
cadences = [3]

for cad in cadences:
    delete_files('lowz', cad)
    hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/LOWZ_{cad}DCAD/LOWZ_{cad}DCAD_HEAD.FITS'
    hdul_phot_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/LOWZ_{cad}DCAD/LOWZ_{cad}DCAD_PHOT.FITS'
    mag_sys = 'BD17'
    sn_name_prefix = f'lowz{cad}dcad_'

    extract_sample(hdul_head_file, hdul_phot_file, mag_sys, sn_name_prefix)

487 SNe extracted!


In [69]:
cad = 7
hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/LOWZ_{cad}DCAD/LOWZ_{cad}DCAD_HEAD.FITS'
fits_file = f'/media/data1/muller/SNANA/test/LOWZ/LOWZ_{cad}DCAD.FITRES.TEXT'
sn_name = '2022'

compare_salt2(hdul_head_file, fits_file, sn_name)

SN: 2022

Simulated SALT2 params --- mb: 16.738, x1: -1.361, dm15: 1.553, c: -0.035, tmax: 54067.141
Fitted SALT2 params    --- mb: 16.716, x1: -1.362, dm15: 1.554, c: -0.047, tmax: 54067.281


In [70]:
bands_list = ['f', 'g', 'h', 'i', 'j']
print_sim_mags(hdul_head_file, sn_name, bands_list)
bands_list = ['k', 'l', 'm', 'n', 'o']
print_sim_mags(hdul_head_file, sn_name, bands_list)

SN: 2022

f = 16.85140037536621
g = 16.94891357421875
h = 16.98889923095703
i = 17.004911422729492
j = 17.505456924438477
SN: 2022

k = -9.0
l = -9.0
m = -9.0
n = -9.0
o = -9.0


___
# **SDSS**

In [9]:
#cadences = [1, 3, 5, 7]
cadences = [3]

for cad in cadences:
    delete_files('sdss', cad)
    hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/SDSS_{cad}DCAD/SDSS_{cad}DCAD_HEAD.FITS'
    hdul_phot_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/SDSS_{cad}DCAD/SDSS_{cad}DCAD_PHOT.FITS'
    mag_sys = 'AB'
    sn_name_prefix = f'sdss{cad}dcad_'

    extract_sample(hdul_head_file, hdul_phot_file, mag_sys, sn_name_prefix)

Ready!


In [12]:
cad = 7
hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/SDSS_{cad}DCAD/SDSS_{cad}DCAD_HEAD.FITS'
fits_file = f'/media/data1/muller/SNANA/test/SDSS/SDSS_{cad}DCAD.FITRES.TEXT'
sn_name = '131'

compare_salt2(hdul_head_file, fits_file, sn_name)

SN: 131

Simulated SALT2 params --- mb: 19.527, x1: 2.027, dm15: 1.287, c: -0.406, tmax: 53652.469
Fitted SALT2 params    --- mb: 19.542, x1: 2.211, dm15: 1.356, c: -0.399, tmax: 53652.695


In [14]:
bands_list = ['g', 'r', 'i']
print_sim_mags(hdul_head_file, sn_name, bands_list)

SN: 131

g = 19.34642791748047
r = 19.744794845581055
i = 20.08390235900879


___
# **SNLS**

In [31]:
#cadences = [1, 3, 5, 7]
cadences = [3]

for cad in cadences:
    delete_files('snls', cad)
    hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/SNLS_{cad}DCAD/SNLS_{cad}DCAD_HEAD.FITS'
    hdul_phot_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/SNLS_{cad}DCAD/SNLS_{cad}DCAD_PHOT.FITS'
    mag_sys = 'AB'
    sn_name_prefix = f'snls{cad}dcad_'

    extract_sample(hdul_head_file, hdul_phot_file, mag_sys, sn_name_prefix)

500 SNe extracted!


In [71]:
cad = 7
hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/SNLS_{cad}DCAD/SNLS_{cad}DCAD_HEAD.FITS'
fits_file = f'/media/data1/muller/SNANA/test/SNLS/SNLS_{cad}DCAD.FITRES.TEXT'
sn_name = '138'

compare_salt2(hdul_head_file, fits_file, sn_name)

SN: 138

Simulated SALT2 params --- mb: 20.161, x1: 3.588, dm15: 2.126, c: -0.413, tmax: 52910.148
Fitted SALT2 params    --- mb: 20.163, x1: 3.592, dm15: 2.129, c: -0.413, tmax: 52910.180


In [19]:
bands_list = ['g', 'r', 'i', 'z']
print_sim_mags(hdul_head_file, sn_name, bands_list)

SN: 57299

g = 19.904033660888672
r = 20.018156051635742
i = 20.180217742919922
z = -9.0


___
# **PS1**

In [32]:
#cadences = [1, 3, 5, 7]
cadences = [3]

for cad in cadences:
    delete_files('ps1', cad)
    hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/PS1_{cad}DCAD/PS1_{cad}DCAD_HEAD.FITS'
    hdul_phot_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/PS1_{cad}DCAD/PS1_{cad}DCAD_PHOT.FITS'
    mag_sys = 'AB'
    sn_name_prefix = f'ps1{cad}dcad_'

    extract_sample(hdul_head_file, hdul_phot_file, mag_sys, sn_name_prefix)

550 SNe extracted!


In [32]:
cad = 7
hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/SIM/PS1_{cad}DCAD/PS1_{cad}DCAD_HEAD.FITS'
fits_file = f'/media/data1/muller/SNANA/test/PS1/PS1_{cad}DCAD.FITRES.TEXT'
sn_name = '88'

compare_salt2(hdul_head_file, fits_file, sn_name)

SN: 88

Simulated SALT2 params --- mb: 23.658, x1: -0.505, dm15: 1.205, c: -0.026, tmax: 55805.473
Fitted SALT2 params    --- mb: 23.680, x1: -1.090, dm15: 1.422, c: -0.038, tmax: 55807.641


In [33]:
bands_list = ['g', 'r', 'i', 'z']
print_sim_mags(hdul_head_file, sn_name, bands_list)

SN: 88

g = 25.13889503479004
r = 23.587730407714844
i = 23.315210342407227
z = 23.416765213012695


___
___
___

In [75]:

hdul_head_file = f'/media/data1/muller/SNANA/SNDATA_ROOT/kcor/SDSS/SNCOSM09+fitsFormat/kcor_SNLS_Bessell90_BD17.fits.gz'

hdul_head = fits.open(hdul_head_file)
column_names = hdul_head[1].data.dtype.names
column_names, hdul_head[1].data

(('Filter Name',
  'Primary Name',
  'Primary Mag',
  'ZPoff(Primary)',
  'ZPoff(SNpot)'),
 FITS_rec([('CFHT-g', 'BD17', 9.7174,  0.1161719 , 0.),
           ('CFHT-r', 'BD17', 9.224 , -0.12163895, 0.),
           ('CFHT-i', 'BD17', 8.905 , -0.3475831 , 0.),
           ('CFHT-z', 'BD17', 8.7543, -0.48531947, 0.),
           ('Bessell-U', 'BD17', 9.724 , -0.75665027, 0.),
           ('Bessell-B', 'BD17', 9.907 ,  0.13407062, 0.),
           ('Bessell-V', 'BD17', 9.464 ,  0.014176  , 0.),
           ('Bessell-R', 'BD17', 9.166 , -0.15989093, 0.),
           ('Bessell-I', 'BD17', 8.846 , -0.39927658, 0.),
           ('Bessell-BX', 'BD17', 9.907 ,  0.14380072, 0.)],
          dtype=(numpy.record, [('Filter Name', 'S20'), ('Primary Name', 'S20'), ('Primary Mag', '>f4'), ('ZPoff(Primary)', '>f4'), ('ZPoff(SNpot)', '>f4')])))